In [1]:
import pandas as pd
from Utilities.references import extract_refs, read_refs

In [ ]:
# ref_xlsx_path = './data/references/reference.xlsx'
# extract_refs(ref_xlsx_path)

# ref_path = './data/references/references.json'
# references = read_refs(ref_path)

# data_path = './data/road_casualty_dataset.csv'
# data_df = pd.read_csv(data_path)

In [11]:
data_path = './data/road_casualty_dataset.csv'

# Importing the dataset and removing unnecessary columns
data_df = pd.read_csv(data_path, na_values=['-1', -1]).reset_index()\
    .drop(columns=['index', 'status', 'accident_year',
                   'accident_reference', 'casualty_reference',
                   'casualty_type', 'lsoa_of_casualty'])

In [12]:
data_df.isnull().sum()

accident_index                           0
vehicle_reference                        0
casualty_class                           0
sex_of_casualty                        448
age_of_casualty                       1350
age_band_of_casualty                  1350
casualty_severity                        0
pedestrian_location                      0
pedestrian_movement                      0
car_passenger                          314
bus_or_coach_passenger                  23
pedestrian_road_maintenance_worker     113
casualty_home_area_type               5498
casualty_imd_decile                   5784
dtype: int64

In [14]:
data_df = data_df.dropna(subset=['casualty_imd_decile', 'casualty_home_area_type'])
data_df.isnull().sum()

accident_index                          0
vehicle_reference                       0
casualty_class                          0
sex_of_casualty                       346
age_of_casualty                       789
age_band_of_casualty                  789
casualty_severity                       0
pedestrian_location                     0
pedestrian_movement                     0
car_passenger                         246
bus_or_coach_passenger                 11
pedestrian_road_maintenance_worker     99
casualty_home_area_type                 0
casualty_imd_decile                     0
dtype: int64

In [ ]:
from sklearn.linear_model import LinearRegression

# Assuming df is your DataFrame and 'target' is the column with missing values
train_df = df[df['target'].notnull()]
predict_df = df[df['target'].isnull()]

# Split into X (features) and y (target)
X_train = train_df.drop('target', axis=1)
y_train = train_df['target']
X_predict = predict_df.drop('target', axis=1)

# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict the missing values
y_predict = model.predict(X_predict)

# Fill the missing values in the original DataFrame
df.loc[df['target'].isnull(), 'target'] = y_predict
